# Models

> Fill in a module description here


In [ ]:
#| default_exp learner_utils

In [ ]:
#| hide
from nbdev.showdoc import *  # type: ignore # noqa: F403

In [ ]:
#| export
import os
import torch.nn as nn
import torch
from fedai.vision.VisionBlock import VisionBlock
from fedai.vision.models import *
from fedai.text.models import *
from fedai.models import * # noqa: F403
from peft import *  # type: ignore # noqa: F403


In [ ]:
#| export
def get_block(cfg, id, train=True):
    block = VisionBlock if cfg.data.modality == ['Vision'] else None
    return block(cfg, id, train=train)

In [ ]:
#| export
def get_model(cfg):
    model_name = cfg.model.name

    # Check if the model name contains "hf://"
    if model_name.startswith("hf://"):
        return get_hf_model(cfg)  # type: ignore # Call your HF model loader function  # noqa: F405

    # Define the rest of the model mapping
    mapping = {
        "MNISTCNN": MNISTCNN(num_classes=10),  # noqa: F405 # type: ignore
        "CIFAR10CNN": CIFAR10CNN(num_classes=10),  # noqa: F405 # type: ignore
        
        "MLP": MLP(  # noqa: F405
            dim_in=cfg.model.dim_in, 
            dim_hidden=cfg.model.dim_hidden, 
            dim_out=cfg.model.dim_out
            ),

        "CharacterLSTM": CharacterLSTM(  # noqa: F405 # type: ignore
            vocab_size=cfg.model.vocab_size,
            embed_size=cfg.model.embed_size,
            hidden_size=cfg.model.hidden_size,
            num_layers=cfg.model.num_layers
        )
    }

    # Look up the model in the mapping
    if model_name in mapping:
        return mapping[model_name]
    
    raise ValueError(f"Model '{model_name}' is not recognized, the available models are: {list(mapping.keys())}")


In [ ]:
#| export
def get_criterion(customm_fn):
    if customm_fn:
        return customm_fn
    else:
        raise ValueError("No criterion function provided")

In [ ]:
#| export
def load_state_from_disk(cfg, model, id, comm_round):
    
    model_path = os.path.join(cfg.save_dir,
                              str(comm_round),
                              f"local_output_{id}",
                              "pytorch_model.bin")

    if os.path.exists(model_path):
        if isinstance(model, torch.nn.Module):
            model.load_state_dict(torch.load(model_path, map_location= model))
        else:
            set_peft_model_state_dict(model,  # noqa: F405 # type: ignore
                                  torch.load(model_path, map_location= model.device), 
                                  "default")
    return model

In [ ]:
#| export
def client_fn(client_cls, cfg, id, latest_round, loss_fn):
    model = get_model(cfg)
    criterion = get_criterion(loss_fn)
    # get train and test ds
    train_block, test_block = get_block(cfg, id), get_block(cfg, id, train=False)

    state = {'model': model, 'optimizer': None, 'criterion': criterion}

    if id in latest_round:
        comm_round = latest_round[id]
        state['model'] = load_state_from_disk(cfg, model, id, comm_round)  # noqa: F405
    
    return client_cls(id, cfg, state, block= [train_block, test_block])


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # type: ignore  # noqa: E702
